#### Importing required Modules and Libraries

In [1]:
import re
import nltk
import gensim
import pyLDAvis.gensim
import pyLDAvis.sklearn
from gensim import corpora
import gensim.downloader as api
from nltk.corpus import stopwords
from sklearn.decomposition import NMF
from gensim.utils import simple_preprocess, lemmatize
from gensim.models import LdaModel , LdaMulticore,LsiModel

import warnings
warnings.filterwarnings('ignore')

In [2]:
#downloading and import The Data
print(api.info("text8"))
dataset = api.load("text8")
data = [d for d in dataset]

{'num_records': 1701, 'record_format': 'list of str (tokens)', 'file_size': 33182058, 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/text8/__init__.py', 'license': 'not found', 'description': 'First 100,000,000 bytes of plain text from Wikipedia. Used for testing purposes; see wiki-english-* for proper full Wikipedia datasets.', 'checksum': '68799af40b6bda07dfa47a32612e5364', 'file_name': 'text8.gz', 'read_more': ['http://mattmahoney.net/dc/textdata.html'], 'parts': 1}


In [3]:
print(" ".join(data[2][:300]))

with the aegis of zeus when he goes to the battlefield the entire trojan army flees behind the walls of troy achilles wrath is terrible and he slays many trojan warriors and allies including priam s son lycaon whom achilles had previously captured and sold into slavery but who had been returned to troy eventually hector comes out of the walls to defend the honour of troy he asked achilles to agree that the body of the loser would be returned for proper burial by the winner achilles rejected this arrangement saying though twenty ransoms and thy weight in gold were offered i would refuse it all stories tell that hector ran about troy seven times and achilles followed him however seeing that achilles would not be outrun hector stood his ground and fought other versions of the tale say that achilles chased after hector two times and one time he was delivered by the gods however on their second encounter achilles trapped hector and challenged him after a legendary fight achilles kills hecto

In [4]:
#preparing NLTK Stopwords
stop_words = stopwords.words('english')
stop_words = stop_words + ['also','year','new','person','many','first']

In [5]:
## Preprocessing Data , lemmatizing , splitting each word and deleting words from stopword
data_processed = []
for i,doc in enumerate(data[:100]):
    doc_out=[]
    for x in doc:
        if x not in stop_words:
            #using only noun ,verb, adjectives and adverbs
            lemma = lemmatize(x,allowed_tags=re.compile('(NN|JJ|RB)'))
            if lemma:
                doc_out = doc_out + [lemma[0].split(b'/')[0].decode('utf-8')]
        else:
            continue
#             data_processed.append(doc_out) doc_out , splitting and lemmatization
    data_processed.append(doc_out)
data_processed

[['anarchism',
  'originated',
  'term',
  'abuse',
  'early',
  'class',
  'radical',
  'digger',
  'english',
  'revolution',
  'san',
  'culotte',
  'french',
  'revolution',
  'term',
  'still',
  'pejorative',
  'way',
  'act',
  'violent',
  'organization',
  'society',
  'positive',
  'label',
  'self',
  'defined',
  'anarchist',
  'word',
  'anarchism',
  'greek',
  'archon',
  'ruler',
  'chief',
  'king',
  'anarchism',
  'political',
  'philosophy',
  'belief',
  'ruler',
  'unnecessary',
  'interpretation',
  'anarchism',
  'related',
  'social',
  'movement',
  'elimination',
  'authoritarian',
  'institution',
  'particularly',
  'state',
  'word',
  'anarchy',
  'anarchist',
  'use',
  'imply',
  'chaos',
  'nihilism',
  'rather',
  'harmonious',
  'authoritarian',
  'society',
  'place',
  'authoritarian',
  'political',
  'structure',
  'coercive',
  'economic',
  'institution',
  'anarchist',
  'social',
  'relation',
  'voluntary',
  'association',
  'autonomous',
 

In [6]:
#Creating the Dictionary and Corpus needed for Topic Modeling
dct = corpora.Dictionary(data_processed)
corpus = [dct.doc2bow(line) for line in data_processed]
print(corpus[:1])

[[(0, 4), (1, 7), (2, 1), (3, 1), (4, 2), (5, 2), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 6), (20, 12), (21, 4), (22, 1), (23, 1), (24, 7), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 4), (31, 2), (32, 3), (33, 2), (34, 1), (35, 1), (36, 1), (37, 1), (38, 7), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 4), (45, 3), (46, 1), (47, 1), (48, 1), (49, 1), (50, 3), (51, 2), (52, 2), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 2), (59, 1), (60, 6), (61, 1), (62, 2), (63, 4), (64, 9), (65, 102), (66, 145), (67, 1), (68, 3), (69, 33), (70, 1), (71, 16), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 2), (78, 1), (79, 2), (80, 1), (81, 1), (82, 3), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1), (90, 3), (91, 2), (92, 1), (93, 2), (94, 1), (95, 1), (96, 1), (97, 2), (98, 2), (99, 2), (100, 5), (101, 1), (102, 1), (103, 1), (104, 6), (105, 2), (106, 1), (107, 6), (108, 3), (109, 2), (

In [7]:
# understandable format of corpus (term-frequency)
[[(dct[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ability', 4),
  ('able', 7),
  ('abnormal', 1),
  ('abolition', 1),
  ('absence', 2),
  ('absolute', 2),
  ('abstention', 1),
  ('absurdity', 1),
  ('abundance', 1),
  ('abuse', 2),
  ('academic', 1),
  ('access', 2),
  ('accidentally', 1),
  ('accommodation', 1),
  ('accordance', 1),
  ('account', 1),
  ('accurately', 1),
  ('accusation', 1),
  ('achievement', 2),
  ('act', 6),
  ('action', 12),
  ('active', 4),
  ('activism', 1),
  ('activist', 1),
  ('activity', 7),
  ('adam', 1),
  ('address', 1),
  ('adherent', 1),
  ('adjective', 1),
  ('adjust', 1),
  ('adult', 4),
  ('advance', 2),
  ('advocate', 3),
  ('affected', 2),
  ('affection', 1),
  ('affinity', 1),
  ('african', 1),
  ('afterwards', 1),
  ('age', 7),
  ('aggression', 2),
  ('aggressive', 1),
  ('agitation', 1),
  ('agitator', 1),
  ('agricultural', 1),
  ('aid', 4),
  ('aide', 3),
  ('aids', 1),
  ('alfredo', 1),
  ('alfredsson', 1),
  ('alienation', 1),
  ('alliance', 3),
  ('almost', 2),
  ('alone', 2),
  ('alrea

#### Building the Topic Model

In [8]:
## Lda model 
lda_model = LdaModel(corpus=corpus,
                     id2word=dct,
                     num_topics=20,
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto',
                     per_word_topics=True)

In [9]:
lda_model.save('lda_model.model')
lda_model.print_topics(-1)

[(0,
  '0.006*"american" + 0.005*"australia" + 0.005*"state" + 0.004*"time" + 0.003*"australian" + 0.003*"einstein" + 0.003*"hiv" + 0.003*"albert" + 0.003*"world" + 0.003*"language"'),
 (1,
  '0.010*"atheism" + 0.007*"god" + 0.006*"atheist" + 0.004*"turing" + 0.004*"church" + 0.004*"albatross" + 0.004*"belief" + 0.003*"newton" + 0.003*"book" + 0.003*"character"'),
 (2,
  '0.007*"africa" + 0.007*"state" + 0.007*"war" + 0.006*"north" + 0.006*"angola" + 0.005*"south" + 0.005*"alberta" + 0.005*"union" + 0.004*"ocean" + 0.004*"character"'),
 (3,
  '0.019*"apple" + 0.011*"aol" + 0.007*"albatross" + 0.006*"computer" + 0.005*"austria" + 0.005*"austrian" + 0.004*"user" + 0.004*"alabaster" + 0.004*"service" + 0.004*"mac"'),
 (4,
  '0.012*"attila" + 0.011*"amsterdam" + 0.008*"hun" + 0.006*"city" + 0.005*"roman" + 0.004*"sea" + 0.004*"empire" + 0.004*"alex" + 0.004*"axiom" + 0.004*"set"'),
 (5,
  '0.011*"jews" + 0.007*"adelaide" + 0.006*"american" + 0.006*"jewish" + 0.005*"semitism" + 0.005*"state

In [10]:
from gensim.models import CoherenceModel
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_processed, dictionary=dct, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.136452924736064

Coherence Score:  0.33591712873310836


In [11]:
#These topics can be classified into:--
    # topic 0,6 are more over same , about history and war stuff
    # topic 3 is fully based on apollo moon mission
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda_model,corpus,dct)
panel

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.073671  0.029065       1        1  13.131504
10     0.063094  0.041877       2        1  10.364317
9      0.070107 -0.073114       3        1   8.884448
7      0.052874 -0.008652       4        1   8.667977
18     0.050799 -0.037474       5        1   7.323925
13     0.050403 -0.043907       6        1   7.059038
1      0.041748 -0.110445       7        1   6.425729
2      0.065470  0.065879       8        1   5.764762
5      0.048534  0.051697       9        1   5.172412
6      0.040009  0.010295      10        1   4.643002
15     0.038054 -0.047284      11        1   4.346971
19    -0.001317  0.147371      12        1   3.527353
17    -0.000616  0.016842      13        1   3.002758
11     0.014407 -0.026548      14        1   2.977576
8     -0.023808  0.030379      15        1   2.820486
3     -0.039792 -0.025658      16        1   2.075449
12    -0.116990 -0.010346      17        1   1.062700
16    -0.144948 -0.006995      18        1   1.032416
4     -0.115929  0.033372      19        1   0.997842
14    -0.165771 -0.036353      20        1   0.719337, topic_info=             Term         Freq        Total Category  logprob  loglift
12688       agave   533.000000   533.000000  Default  30.0000  30.0000
1759       apollo   361.000000   361.000000  Default  29.0000  29.0000
3424      lincoln   345.000000   345.000000  Default  28.0000  28.0000
9545        apple   255.000000   255.000000  Default  27.0000  27.0000
60       american  1745.000000  1745.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
972         music     7.413194   248.308289  Topic20  -6.0072   1.4232
827          king     7.778539   513.348145  Topic20  -5.9591   0.7450
2874   university     7.082639   449.406403  Topic20  -6.0528   0.7843
678          hand     6.926129   245.587402  Topic20  -6.0752   1.3662
6195     computer     6.769297   344.919983  Topic20  -6.0981   1.0037

[1450 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
27597     12  0.950303   aalborg
12677      2  0.961114  aardvark
27601     12  0.959850    aarhus
25596      2  0.011236     aaron
25596      9  0.022472     aaron
...      ...       ...       ...
14022      1  0.088916   zionism
14022      8  0.088916   zionism
14022      9  0.800243   zionism
20372     19  0.829117      zorn
20373     19  0.446585      zuid

[6860 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 11, 10, 8, 19, 14, 2, 3, 6, 7, 16, 20, 18, 12, 9, 4, 13, 17, 5, 15])

In [12]:
# We will try using LSI model - Latent semantic indexing
lsi_model = LsiModel(corpus=corpus,
                     id2word=dct,
                     num_topics=20,
                     chunksize=100,)

In [13]:
lsi_model.print_topics(-1)

[(0,
  '0.218*"american" + 0.217*"state" + 0.161*"time" + 0.141*"world" + 0.137*"war" + 0.117*"person" + 0.111*"agave" + 0.108*"history" + 0.100*"number" + 0.099*"language"'),
 (1,
  '-0.935*"agave" + -0.163*"asia" + -0.100*"aruba" + -0.062*"plant" + -0.053*"var" + 0.048*"american" + -0.044*"state" + -0.044*"east" + -0.043*"congress" + -0.041*"maguey"'),
 (2,
  '0.501*"american" + 0.195*"football" + 0.189*"player" + 0.164*"war" + 0.153*"british" + 0.126*"ball" + -0.113*"god" + 0.111*"play" + -0.110*"atheism" + 0.110*"line"'),
 (3,
  '-0.329*"apollo" + 0.229*"lincoln" + 0.201*"state" + -0.191*"player" + -0.174*"football" + -0.148*"ball" + -0.147*"line" + -0.128*"play" + 0.127*"government" + 0.126*"union"'),
 (4,
  '-0.363*"atheism" + -0.344*"lincoln" + -0.334*"god" + -0.215*"atheist" + -0.212*"apollo" + -0.147*"abraham" + -0.136*"aristotle" + 0.134*"island" + 0.123*"aluminium" + -0.120*"belief"'),
 (5,
  '0.426*"lincoln" + -0.360*"atheism" + -0.269*"god" + 0.239*"apollo" + -0.214*"athei

#### Building the Topic Model only using Noun words this time

In [14]:
data_processed = []
for i,doc in enumerate(data[:50]):
    doc_out=[]
    for x in doc:
        if x not in stop_words:
            lemma = lemmatize(x,allowed_tags=re.compile('(NN)'))
            if lemma:
                doc_out = doc_out + [lemma[0].split(b'/')[0].decode('utf-8')]
        else:
            continue
    data_processed.append(doc_out)

In [15]:
dct = corpora.Dictionary(data_processed)
corpus = [dct.doc2bow(line) for line in data_processed]

In [16]:
lda_model = LdaModel(corpus=corpus,
                     id2word=dct,
                     num_topics=20,
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='asymmetric',
                     per_word_topics=True)

In [17]:
lda_model.print_topics(-1)

[(0,
  '0.015*"lincoln" + 0.014*"aristotle" + 0.009*"alchemy" + 0.009*"ad" + 0.006*"clarke" + 0.006*"science" + 0.005*"abraham" + 0.005*"philosophy" + 0.005*"time" + 0.005*"work"'),
 (1,
  '0.017*"section" + 0.014*"taggart" + 0.014*"atla" + 0.011*"rearden" + 0.010*"dagny" + 0.010*"state" + 0.010*"world" + 0.008*"galt" + 0.008*"person" + 0.008*"anthropology"'),
 (2,
  '0.011*"atom" + 0.007*"element" + 0.007*"animation" + 0.006*"number" + 0.006*"electron" + 0.005*"arsenic" + 0.005*"god" + 0.004*"time" + 0.004*"province" + 0.004*"set"'),
 (3,
  '0.023*"africa" + 0.011*"character" + 0.011*"afghanistan" + 0.009*"country" + 0.009*"god" + 0.007*"language" + 0.007*"austin" + 0.006*"name" + 0.006*"continent" + 0.006*"code"'),
 (4,
  '0.016*"angola" + 0.012*"anthropology" + 0.009*"archaeology" + 0.007*"culture" + 0.006*"state" + 0.006*"science" + 0.006*"site" + 0.006*"study" + 0.006*"schopenhauer" + 0.005*"world"'),
 (5,
  '0.022*"language" + 0.009*"vs" + 0.008*"president" + 0.008*"bush" + 0.008

In [19]:
from gensim.models import CoherenceModel
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_processed, dictionary=dct, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.257294462850423

Coherence Score:  0.3403933126037372


In [18]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda_model,corpus,dct)
pyLDAvis.display(panel)